In [1]:
!pip install accelerate

In [4]:
# AutoModelForCausalLM 是用于加载预训练的因果语言模型（如GPT系列）
# 而 AutoTokenizer 是用于加载与这些模型匹配的分词器。
from transformers import AutoModelForCausalLM, AutoTokenizer
model_dir = "Qwen/Qwen1.5-14B-Chat"
device = "cuda"

# torch_dtype="auto" 自动选择最优的数据类型以平衡性能和精度。
# device_map="auto" 自动将模型的不同部分映射到可用的设备上。
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype="auto",
    device_map="auto"
)

# 加载与模型相匹配的分词器。分词器用于将文本转换成模型能够理解和处理的格式。
tokenizer = AutoTokenizer.from_pretrained(model_dir)

prompt = "你好，请介绍下你自己。"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# 使用分词器的 apply_chat_template 方法将上面定义的消息列表转换为一个格式化的字符串，适合输入到模型中。
# tokenize=False 表示此时不进行令牌化，add_generation_prompt=True 添加生成提示。
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 将处理后的文本令牌化并转换为模型输入张量，然后将这些张量移至之前定义的设备（GPU）上。
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# 使用模型的 generate 方法基于输入生成文本。
# max_new_tokens=512 限制了生成的最大令牌数量，以控制输出长度。
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)

# 从生成的令牌中提取新生成的部分，忽略输入部分的令牌。
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# 将生成的令牌ID解码为人类可读的文本，并且跳过特殊令牌，如填充令牌或其他非文本内容的标记。
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)